In [1]:
import torch
import os
from pandas import read_csv
from torch.optim import SGD
from torch.utils.data import DataLoader
from tensorboardX import SummaryWriter
from config import DATA_DIR, MODELS_DIR
from src.loss import Loss
from src.model import HistogramMF
from src.runner import Runner
from src.create_dataset import create_dataset
from src.data_processor import DataProcessor
from src.data_encoder import DataEncoder

DF_PATH = f"{DATA_DIR}/BookCrossing/BX-Book-Ratings-With-Histogram_features.csv"

In [2]:
columns = ["user_id", "item_id", "rating", "original_mass", "total_mass"]
original_df = read_csv(DF_PATH, skipinitialspace=True, names=columns)
original_df = original_df.iloc[1:, :]

original_df = original_df.astype({"user_id": "int32"})
original_df = original_df.astype({"item_id": str})
original_df = original_df.astype({"rating": "int32"})
original_df = original_df.astype({"original_mass": "float64"})
original_df = original_df.astype({"total_mass": "float64"})

/var/folders/tf/cy7jlqss4qs8lktmfhln3y5w0000gq/T/ipykernel_18512/2168540578.py:2: DtypeWarning: Columns (1,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  original_df = read_csv(DF_PATH, skipinitialspace=True, names=columns)


In [3]:
data_encoder = DataEncoder(original_df=original_df)
data_processor = DataProcessor(original_df=original_df)

n_users = original_df.user_id.nunique()
n_items = original_df.item_id.nunique()

min_rating = min(original_df.rating.values)
max_rating = max(original_df.rating.values)

model = HistogramMF(
    n_users=n_users,
    n_items=n_items,
    data_encoder=data_encoder,
    data_processor=data_processor,
    min_rating=min_rating,
    max_rating=max_rating,
)

if os.path.exists(f"{MODELS_DIR}/book_crossing/model.pt"):
    model.load_state_dict(torch.load(f"{MODELS_DIR}/book_crossing/model.pt"))

epochs = 30

criterion = Loss()
optimizer = SGD(model.parameters(), lr=5, weight_decay=1e-7)
runner = Runner(model=model, criterion=criterion, optimizer=optimizer)

train_set = create_dataset(data_encoder=data_encoder)
train_load = DataLoader(train_set, batch_size=1000, shuffle=True)

with SummaryWriter(f"runs/book_crossing/dev") as writer:
    for epoch in range(epochs):
        epoch_loss = runner.train(train_loader=train_load, epoch=epoch, writer=writer)
        print(f"epoch={epoch + 1}, loss={epoch_loss}")
        torch.save(model.state_dict(), f"{MODELS_DIR}/book_crossing/model.pt")

100%|██████████| 1150/1150 [03:56<00:00,  4.86batch/s, train_loss=0.107] 


epoch=1, loss=126.46271296398457


100%|██████████| 1150/1150 [04:14<00:00,  4.52batch/s, train_loss=0.108] 


epoch=2, loss=116.80160687568862


100%|██████████| 1150/1150 [04:07<00:00,  4.65batch/s, train_loss=0.0976]


...

epoch=28, loss=39.49946111527469


100%|██████████| 1150/1150 [04:01<00:00,  4.76batch/s, train_loss=0.0375]


epoch=29, loss=38.837134739998085


100%|██████████| 1150/1150 [04:02<00:00,  4.75batch/s, train_loss=0.0329]

epoch=30, loss=38.1259297607129


In [4]:
from pandas import DataFrame
from collections import namedtuple

Row = namedtuple("Row", "user_id item_id rating")

data_encoder = DataEncoder(original_df=original_df)
dataframe_after_mf = []

with torch.no_grad():
    for (index, user_id, item_id, rating, _, _) in original_df.itertuples():
        encoded_user_id = data_encoder.get_encoded_user_id(original_id=user_id)
        encoded_item_id = data_encoder.get_encoded_item_id(original_id=item_id)

        user_id_as_tensor = torch.LongTensor([encoded_user_id])
        item_id_as_tensor = torch.LongTensor([encoded_item_id])
        output = model(users=user_id_as_tensor, items=item_id_as_tensor,).squeeze()[0]
        predicted_rating = torch.round(output).item()

        dataframe_after_mf.append(Row(user_id=user_id, item_id=item_id, rating=predicted_rating))

df_after_mf = DataFrame(dataframe_after_mf, columns=["user_id", "item_id", "rating"])
df_after_mf.head()

,user_id,item_id,rating
0,276725,034545104X,0.0
1,276726,0155061224,3.0
2,276727,0446520802,-0.0
3,276729,052165615X,3.0
4,276729,0521795028,4.0


In [14]:
compare = original_df[["user_id", "item_id", "rating"]].reset_index(drop=True)
df_after_mf = df_after_mf.reset_index(drop=True)
# df_after_mf.head()
mask = (compare["rating"] == df_after_mf["rating"])
changes = compare[mask].copy()
changes["New rating"] = df_after_mf.rating
print(f"Number of hits: {len(changes)} / {len(original_df)}")
changes.head(len(changes))

Number of hits: 1061273 / 1149780


,user_id,item_id,rating,New rating
0,276725,034545104X,0,0.0
2,276727,0446520802,0,-0.0
3,276729,052165615X,3,3.0
8,276744,038550120X,7,7.0
10,276746,0425115801,0,-0.0
...,...,...,...,...
1149773,276704,0806917695,5,5.0
1149774,276704,0876044011,0,0.0
1149775,276704,1563526298,9,9.0
1149776,276706,0679447156,0,-0.0
